# Spark environment

In [2]:
import pandas as pd
import numpy as np
import math 
import os
from ast import literal_eval
from pathlib import Path
from scipy import spatial
from numpy import dot
from numpy.linalg import norm
import datetime
import matplotlib.pyplot as plt
from numpy import radians, cos, sin, arcsin, sqrt
from math import sin, cos, acos, radians, asin, sqrt
from functools import reduce
import gensim
from gensim.models import Word2Vec
from gensim.test.utils import common_texts
from gensim.models import Phrases
from nltk.tokenize import sent_tokenize, word_tokenize
import findspark
import pyspark.sql.functions as f
from pyspark.sql import DataFrame
from pyspark.sql.functions import split, col, udf, size, lit
from pyspark.sql.types import StringType
from urllib import request
from IPython.core.magic import (register_line_magic, register_cell_magic, register_line_cell_magic)
import zipfile
import warnings
warnings.filterwarnings(action = 'ignore')
%matplotlib inline

In [7]:
home = os.environ["HOME"]

D1 = "/.local/lib/python3.9/site-packages/pyspark"
D2 = "/root/anaconda3/lib/python3.9/site-packages/pyspark"

if os.path.exists(D1):
    os.environ["SPARK_HOME"] = home +  D1
elif os.path.exists(D2):
    os.environ["SPARK_HOME"] = home +  D2
else:
    print("PB de path")

os.environ["JAVA_HOME"] ="/usr"

PB de path


### Start spark 

In [8]:
print("findspark.init() initialise les variables d'environnement pour spark") 
findspark.init() 

from pyspark.sql import SparkSession 
from pyspark import SparkConf

findspark.init() initialise les variables d'environnement pour spark


In [10]:
def demarrer_spark():
    local = "local[*]"
    appName = "TP"
    configLocale = SparkConf().setAppName(appName).setMaster(local).\
    set("spark.executor.memory", "100G").\
    set("spark.driver.memory","50G").\
    set("spark.sql.catalogImplementation","in-memory").\
    set("spark.driver.maxResultSize", "10G")

    spark = SparkSession.builder.config(conf = configLocale).getOrCreate()
    sc = spark.sparkContext
    sc.setLogLevel("ERROR")

    # spark.conf.set("spark.sql.autoBroadcastJoinThreshold","-1")
    # On ajuste l'environnement d'exécution des requêtes à la taille du cluster (4 coeurs)
    # spark.conf.set("spark.sql.shuffle.partitions","200")    

    print("session démarrée, son id est ", sc.applicationId)
    return spark

spark = demarrer_spark()

session démarrée, son id est  local-1682975293471


In [11]:
spark

###  SQL Magic

In [12]:
from IPython.core.magic import (register_line_magic, register_cell_magic, register_line_cell_magic)

def removeComments(query):
    result = ""
    for line in query.split('\n'):
    if not(line.strip().startswith("--")):
        result += line + "\n"
    return result

@register_line_cell_magic
def sql(line, cell=None):
    "To run a sql query. Use:  %%sql"
    val = cell if cell is not None else line
    tabRequetes = removeComments(val).split(";")
    derniere = None
    est_requete = False
    for r in tabRequetes:
        r = r.strip()
        if len(r) > 2:
          derniere = spark.sql(r)
          est_requete = r.lower().startswith('select') or r.lower().startswith('with')  
    if(est_requete):
        return display(derniere)
    else:
        return print('ok')

display

In [13]:
def display(df, n=10):
    pd.set_option('max_columns', None)
    pd.set_option('max_colwidth', None)
    return df.limit(n).toPandas()

print("display redéfini")

display redéfini


# Parameters

In [14]:
class Parametre :
    #Dataset storage URL
    PUBLIC_DATASET_URL = "https://nuage.lip6.fr/s/H3bpyRGgnCq2NR4" 


    # Local directory
    local_dir = "/local/jarrad/bd/"

    # models storage directory
    model_dir  = local_dir + "model_dimension"
    model_name = model_dir + "/model" 

    yfcc_web_dir = "YFCC_dataset_extrait"
    yfcc_file = "yfccFrance"
    
    
    # Notebook Directory
    notebook_dir = "/home/jarrad/Bureau/notebook_sara/"


    # experiment_directory
    experiment_dir = notebook_dir + "experiment/"


    # parameters values chosen
    maillage = [10, 40, 70, 150, 300]
    expe_dimensions = [20, 25]
    expe_topk = [15, 20]
    expe_epochs = [100]
    expe_windows = [5]
   
P = Parametre()
print("local_dir:", P.local_dir)
print("model_dir:", P.model_dir)
print("notebook_dir:", P.notebook_dir)
print("experiment_dir:", P.experiment_dir)

local_dir: /local/jarrad/bd/
model_dir: /local/jarrad/bd/model_dimension
notebook_dir: /home/jarrad/Bureau/notebook_sara/
experiment_dir: /home/jarrad/Bureau/notebook_sara/experiment/


In [15]:
os.makedirs(P.local_dir, exist_ok=True)
os.makedirs(P.model_dir, exist_ok=True)
os.makedirs(P.experiment_dir, exist_ok=True)

# Dataset

## Dataset Analysis

In [16]:
os.listdir(P.local_dir)

['model_dimension',
 'yfccFrance_M_40_trainset',
 'yfccFrance_M_30_testset',
 'yfccFrance_testset',
 'yfccFrance_trainset.tgz',
 'yfccFrance_testset.tgz',
 'yfccFrance_M_100_testset',
 'yfccFrance_M_70_trainset',
 'yfccFrance_M_30_trainset',
 'yfccFrance_M_1_trainset',
 'yfccFrance_M_10_testset',
 'yfccFrance_M_40_testset',
 'yfccFrance_M_100_trainset',
 'yfccFrance_M_1_testset',
 'yfccFrance_M_70_testset',
 'yfccFrance_M_5_trainset',
 'yfccFrance_M_200_trainset',
 'yfccFrance_M_150_testset',
 'yfccFrance_M_5_testset',
 'yfccFrance.zip',
 'yfccFrance_M_400_trainset',
 'yfccFrance_M_50_testset',
 'yfccFrance_M_50_trainset',
 'yfccFrance_M_500_trainset',
 'yfccFrance_M_200_testset',
 'yfccFrance_M_1e-07_trainset',
 'yfccFrance_trainset',
 'yfccFrance_M_1e-07_testset',
 'yfccFrance_M_400_testset',
 'yfccFrance_M_20_testset',
 'yfccFrance_M_10_trainset',
 'yfccFrance_M_20_trainset',
 'yfccFrance_M_150_trainset',
 'yfccFrance_M_500_testset',
 'yfccFrance']

Fonction download file

In [17]:
# download dataset if not already donwloaded
def download_file(web_file, local_file):

    if(os.path.isfile(local_file)):
    print(local_file, "is already stored")
    else:
    print("downloading from URL: ", web_file , "save in : " + local_file)
    request.urlretrieve(web_file , local_file)
    
print("définie")

définie


URL to access to dataset

download YFCC file

### YFCC Dataset

### get_yfcc_dataset

In [18]:
import zipfile

def get_yfcc_dataset(web_dir, local_dir, file):

    local_file = local_dir + "/" + file
    web_file = web_dir + "/" + file
    download_file(web_file, local_file + ".zip")

    #unzip
    if(os.path.isdir(local_file)):
        print("file already unziped")
    else:
      with zipfile.ZipFile(local_dir + "/" + file + ".zip", 'r') as zip_ref:
        zip_ref.extractall(local_dir)


call get_yfcc_dataset

In [19]:
print("URL du dossier contenant les datasets ", P.PUBLIC_DATASET_URL)

yfcc_web_dir_download_URL = P.PUBLIC_DATASET_URL + "/download?path=" + "/" + P.yfcc_web_dir
get_yfcc_dataset(yfcc_web_dir_download_URL , P.local_dir, P.yfcc_file)

os.listdir(P.local_dir)

URL du dossier contenant les datasets  https://nuage.lip6.fr/s/H3bpyRGgnCq2NR4
/local/jarrad/bd//yfccFrance.zip is already stored
file already unziped


['model_dimension',
 'yfccFrance_M_40_trainset',
 'yfccFrance_M_30_testset',
 'yfccFrance_testset',
 'yfccFrance_trainset.tgz',
 'yfccFrance_testset.tgz',
 'yfccFrance_M_100_testset',
 'yfccFrance_M_70_trainset',
 'yfccFrance_M_30_trainset',
 'yfccFrance_M_1_trainset',
 'yfccFrance_M_10_testset',
 'yfccFrance_M_40_testset',
 'yfccFrance_M_100_trainset',
 'yfccFrance_M_1_testset',
 'yfccFrance_M_70_testset',
 'yfccFrance_M_5_trainset',
 'yfccFrance_M_200_trainset',
 'yfccFrance_M_150_testset',
 'yfccFrance_M_5_testset',
 'yfccFrance.zip',
 'yfccFrance_M_400_trainset',
 'yfccFrance_M_50_testset',
 'yfccFrance_M_50_trainset',
 'yfccFrance_M_500_trainset',
 'yfccFrance_M_200_testset',
 'yfccFrance_M_1e-07_trainset',
 'yfccFrance_trainset',
 'yfccFrance_M_1e-07_testset',
 'yfccFrance_M_400_testset',
 'yfccFrance_M_20_testset',
 'yfccFrance_M_10_trainset',
 'yfccFrance_M_20_trainset',
 'yfccFrance_M_150_trainset',
 'yfccFrance_M_500_testset',
 'yfccFrance']

In [20]:
def lire_yfcc(nom_fichier):
    #local_dir = "/data/bd/jarrad/"
    local_dir = "/local/jarrad/bd"
    res = spark.read.parquet(local_dir + "/" + nom_fichier).persist()
    return res

yfcc = lire_yfcc(P.yfcc_file)
display(yfcc)

,Line,PhotoID,PhotoHash,UserNSID,UserNickname,DateTaken,DateUploaded,CaptureDevice,Title,Description,UserTags,MachineTags,Longitude,Latitude,Accuracy,URL,DownloadURL,LicenseName,LicenseURL,ServerID,FarmID,Secret,SecretOriginal,Extension,Marker
0,80096796,834796,dc9b7584ecb34a448540bee3b38fe85c,77922700@N00,iko,2004-09-15 19:41:25.0,1097582515,PENTAX+Corporation+PENTAX+Optio+S4,marseille,None,"city,cityproject,coucher+de+soleil,marseille,sunset,ville",None,5.371059,43.284172,16,http://www.flickr.com/photos/77922700@N00/834796/,http://farm1.staticflickr.com/1/834796_9e5d1edb3a.jpg,Attribution-NonCommercial-NoDerivs License,http://creativecommons.org/licenses/by-nc-nd/2.0/,1,1,9e5d1edb3a,9e5d1edb3a,jpg,0
1,64666153,5477598,4c38199a877fba534860a365bf9c97,76384935@N00,Chip_2904,2005-02-26 20:36:16.0,1109450176,None,Quiz+Night+2,None,"france,lesarcs,skiing",None,6.832551,45.573586,13,http://www.flickr.com/photos/76384935@N00/5477598/,http://farm1.staticflickr.com/6/5477598_d4ec281653.jpg,Attribution-NonCommercial-ShareAlike License,http://creativecommons.org/licenses/by-nc-sa/2.0/,6,1,d4ec281653,d4ec281653,jpg,0
2,82397565,5975164,2a76868b7eb18ea240d48e2941648582,70408381@N00,scot2342,2004-07-31 10:12:52.0,1110087781,NIKON+E4200,Giverny,Giverny+flowers+Monet+France,"2004,flowers,france,giverny,monet",None,1.524825,49.077106,16,http://www.flickr.com/photos/70408381@N00/5975164/,http://farm1.staticflickr.com/3/5975164_5ebb925aa1.jpg,Attribution-NonCommercial-NoDerivs License,http://creativecommons.org/licenses/by-nc-nd/2.0/,3,1,5ebb925aa1,5ebb925aa1,jpg,0
3,39862899,8060056,987ba5ffd786949496eda3a5fcf89b2,32323502@N00,Julie70,2005-03-31 12:22:18.0,1112338974,SONY+DSC-P150,They+were+all+in+it,There+will+be+always+loving+couples+in+Paris%2C+kissing+each+other.+It+is+good+to+know...+even+if+it+is+no+more+me.+Between+Notre+Dame+and+the+Seine+there+is+a+quiet+alley.%0ASuddenly%2C+my+eyes+falled+on+this+couple%3A+they+were+all+in+the+kiss+and+the+embrace%2C+showing+the+love+and+tenderness+to+each+other.+%0A%0AThat+is+also+one+of+my+prefered+photos.+One+of+the+things+I+liked+most+were+her+sensible+shows+and+how+she+had+to+be+almost+on+tips+of+her+feet+to+arrive+to+him+and+how+much+reciprocity+showed+in+this+image+from+this+middle+aged+couple.%0A%0AI+remember%2C+looking+at+it%2C+as+probably+I+did+when+I+first+looked+at+it%2C+a+certain+night%2C+long+long+time+ago%3A+I+was+40+years+old%2C+and+my+husband+from+wich+I+was+separating+told+me+%22you+are+too+old%2C+no+one+would+want+you+any+more%22.%0A%0AI+met+an+intelligent+and+tender+American%2C+went+to+diner%2C+showing+him+later+%22Paris%22.+%0AA+night%2C+behind+Notre+Dame++I+discovered+I+can+be+regarded+as+woman+and+with+tenderness+again%2C+and+that+all+is+not+lost+as+I+believed+before.,"1-5-fav,100+most+interesting,100+pages,120+of+50000,2,2005,a4,balade,best+in+22000,bigfavs,blog,choice,couples,deux,flickr+most+favorited,flickrfavs,for+valentine+day+2010,france,gens,images,julie+kertesz,julie70,kett%C3%B6,love,march,master%27s+prefered,mc%2307,mes+favoris,meschoix,most+interesting,mostfav,my+favorites,my+prefered,my+preferred,pair,paires,paris,paris+5e,paris+strolls,parisians,parisien,parisienne,parisiens,people,people+in+paris,personnages,photo+julie+kertesz,photo-pots,photography+julie+kertesz,photos,portrait,prefered,quais,rencontr%C3%A9s+%C3%A0+paris,rencontres,seine,someofmyfavorites,street+photo,stroll,tenderness,tendresse,three+years,together,top,top+favorited,top+interesting,topfavs,topvjulie70,two",None,2.346675,48.852306,14,http://www.flickr.com/photos/32323502@N00/8060056/,http://farm1.staticflickr.com/4/8060056_df5ed9e19b.jpg,Attribution-NonCommercial-ShareAlike License,http://creativecommons.org/licenses/by-nc-sa/2.0/,4,1,df5ed9e19b,df5ed9e19b,jpg,0
4,64976931,8916795,b1e9a0bbfd8a221830461d39a6c7e1d3,51035823282@N01,alexdecarvalho,2005-04-08 17:56:02.0,1113079413,Canon+DIGITAL+IXUS+40,Grupo+Corpo,None,"2005,brasil,brazil,dance,france,francebrazil,grupocorpo,paris",None,2.304210,48.

### Visits table

In [21]:
def visite(yfcc):
    yfcc.createOrReplaceTempView("YFCC")
    visite = """
    select photoID as photo, userNSID as user, timestamp(datetaken) as datetime, date(datetaken) as jour, latitude, longitude,  concat(round(latitude,4), '-', round(longitude,4)) as gridID
    from YFCC
    """
    return spark.sql(visite)

In [22]:
visite1 = visite(yfcc)
display(visite1)

,photo,user,datetime,jour,latitude,longitude,gridID
0,834796,77922700@N00,2004-09-15 19:41:25,2004-09-15,43.284172,5.371059,43.2842-5.3711
1,5477598,76384935@N00,2005-02-26 20:36:16,2005-02-26,45.573586,6.832551,45.5736-6.8326
2,5975164,70408381@N00,2004-07-31 10:12:52,2004-07-31,49.077106,1.524825,49.0771-1.5248
3,8060056,32323502@N00,2005-03-31 12:22:18,2005-03-31,48.852306,2.346675,48.8523-2.3467
4,8916795,51035823282@N01,2005-04-08 17:56:02,2005-04-08,48.866394,2.304210,48.8664-2.3042
5,9785366,57366077@N00,2005-03-17 01:18:20,2005-03-17,48.852177,2.348333,48.8522-2.3483
6,15258293,70786516@N00,2005-05-23 12:49:13,2005-05-23,47.866096,1.783304,47.8661-1.7833
7,15470300,77357393@N00,2002-09-19 00:00:00,2002-09-19,43.576412,7.076911,43.5764-7.0769
8,16396565,49691054@N00,2005-04-02 16:32:55,2005-04-02,49.547153,4.864540,49.5472-4.8645
9,17251304,49691054@N00,2005-05-17 15:46:50,2005-05-17,44.466560,1.670951,44.4666-1.671


### VisiteRound(n)
Function specifying the mesh size used in a number n of meters

In [35]:
def round_to_n_meters(degree_coordinate, n):
    # Approximate number of meters for 1 degree close to the equator
    meters = degree_coordinate * 111319.9
    rounded_meters = round(meters / n) * n
    as_degree = rounded_meters / 111319.9
    return as_degree

spark.udf.register("round_to_n_meters", round_to_n_meters, DoubleType())

<function __main__.round_to_n_meters(degree_coordinate, n)>

In [38]:
def visit_round_to_n_meters(visite, n):
    visite.createOrReplaceTempView("visite")
    query = f"""
            select user, jour, datetime, round_to_n_meters(latitude, {n}) as latitude , round_to_n_meters(longitude, {n}) as longitude
            from visite
            order by latitude, longitude;
            """
    return spark.sql(query)

### POIs in France

#### distinct latitudes, longitudes in the dataset, and their frequency

In [69]:
def lat_long_freq(visite_round3):
    visite_round3.createOrReplaceTempView("Visite_Round")
    query = """
            select latitude, longitude, count(*) as freq
            from Visite_Round
            group by latitude, longitude
            order by freq desc        
            """
    return spark.sql(query)

In [72]:
def bounding_box_France(table_LatLongFrance):
    table_LatLongFrance.createOrReplaceTempView("LatLongFrance")
    query = """
            select * 
            from LatLongFrance
            where latitude between  41.29097926752132 and 51.4708053796387
            and longitude between  -6.04365244747813 and 9.8145023996386
            ;
            
            """
    return spark.sql(query)

#### numbering the POIs

In [75]:
def numorer_POI(latLongFrance):
    latLongFrance.createOrReplaceTempView("numPOI")
    query = """
            select  latitude, longitude, row_number() over (order by latitude, longitude) as num
            from numPOI
            order by latitude, longitude
            ;
            
            """
    return spark.sql(query)

### Initial set with POI numbers

In [78]:
def visiteNum(table_numPOI, table_visiteRound3):
    table_numPOI.createOrReplaceTempView("numPOII")
    table_visiteRound3.createOrReplaceTempView("VisiteRoundd3")
    
    query = """
            select v.*, n.num
            from VisiteRoundd3 v, numPOII n
            where v.latitude = n.latitude and v.longitude = n.longitude;

            
            """
    return spark.sql(query)

### Trajectories construction

In [81]:
def get_trajectoires (listeCouples):
    listeCouples.sort()
    liste_poi = [poi for d,poi in listeCouples]
    return get_trajectoires_V3(liste_poi)


def get_trajectoires_V3(liste_poi):    
    results = []
    result = []
    for poi in liste_poi:
        if len(result)==0: 
            result.append(poi)
        elif result[-1] != poi: 
            if not poi in result:
                result.append(poi)
            else:
                if len(result)>1:
                    results.append(result)
                result = [poi]
    if len(result)>1:    
        results.append(result) 
    return results

spark.udf.register("get_trajectoires", get_trajectoires, ArrayType(ArrayType(IntegerType())))

<function __main__.get_trajectoires(listeCouples)>

#### All trajectory sizes

We limit ourselves to trajectories with a SIZE >=3

In [82]:
def toutes_tailles_trajectoires(table_visites):
    table_visites.createOrReplaceTempView("trajectoires_toutes_taille")
    query = """
            select user, jour, explode(get_trajectoires(collect_list( (to_unix_timestamp(datetime), num)))) as trajectoire
            from trajectoires_toutes_taille
            group by user, jour;
            
            """
    return spark.sql(query)

#### Trajectories with size >=3

In [84]:
def trajectoires_sup3(table_traj_toutes_tailles):
    table_traj_toutes_tailles.createOrReplaceTempView("trajectoires")
    query = """
            select *
            from trajectoires
            where where size(trajectoire) >=3
            order by user,jour;
            
            """
    return spark.sql(query)

### train/test construction

#### Create test trajectories

Avoid cold start: 

Eligibility: a test set is created such that for each trajectory, there is at least another one that has the same 2 last POIs

In [95]:
def trajectoire_eligible(table_traj_sup_3):
    table_traj_sup_3.createOrReplaceTempView("table_traj_sup_3")
    query = """
            select DISTINCT t1.trajectoire, t1.user, t1.jour, element_at(t1.trajectoire, -2) as avant_dernier_poi, element_at(t1.trajectoire, -1) as dernier_poi
            from table_traj_sup_3 t1, table_traj_sup_3 t2
            where element_at(t1.trajectoire, -2) = element_at(t2.trajectoire, -2) 
            and element_at(t1.trajectoire, -1) = element_at(t2.trajectoire, -1) 
            and t1.trajectoire <> t2.trajectoire
            """
    return spark.sql(query)

##### Total number of eligible test trajectories for all meshes

In [99]:
liste_toutes_valeurs_compt = []
for n in P.maillage:
    yfcc = lire_yfcc(P.yfcc_file)
    visite1 = visite(yfcc)
    visiteRound = visit_round_to_n_meters(visite1, n)
    frequence = lat_long_freq(visiteRound)
    latlongFrance = bounding_box_France(frequence)
    table_avec_num_POI = numorer_POI(latlongFrance)
    table_visite_avec_num_POI = visiteNum(table_avec_num_POI, visiteRound)
    traj_toutes_tailles = toutes_tailles_trajectoires(table_visite_avec_num_POI)
    trajectoires_superieur_3 = trajectoires_sup3(traj_toutes_tailles)
    
    trajectoires_eligibles = trajectoire_eligible(trajectoires_superieur_3)
    nombre_trajectoire_eligible = trajectoires_eligibles.count()
    
    longueur_moyenne_traj_eligible = (trajectoires_eligibles.select("trajectoire").withColumn("longueur", f.size("trajectoire"))).agg(f.avg("longueur").alias("longueur_moyenne"))
    longueur_moyenne_traj_eligible_pandas = longueur_moyenne_traj_eligible.toPandas()
    longueur_moyenne_par_maillage = longueur_moyenne_traj_eligible_pandas["longueur_moyenne"][0]
    nombre_poi_france = latlongFrance.count()
    
    
    tuple_taille_traj_par_maillage = (n, nombre_poi_france, nombre_trajectoire_eligible, longueur_moyenne_par_maillage)
    liste_toutes_valeurs_compt.append(tuple_taille_traj_par_maillage)
    
columns=['maillage', 'nombre_poi_france', 'nombre_trajectoire_eligible', 'longueur_moyenne_seq_par_maillage'] 
nbr_traj_eligible = pd.DataFrame(liste_toutes_valeurs_compt, columns = columns)
nbr_traj_eligible

,maillage,nombre_poi_france,nombre_trajectoire_eligible,longueur_moyenne_seq_par_maillage
0,10,473296,503,4.972167
1,40,326923,1987,6.240060
2,70,266331,4133,6.058553
3,150,194579,8982,5.409820
4,300,142211,14064,4.771758


sort by date the trajectories that have the same suffix and number them

In [106]:
def trajectoire_eligible_numerotation(trajectoires_eligibles):
    trajectoires_eligibles.createOrReplaceTempView("trajectoires_eligibles")
    query = """
            select user, jour, trajectoire, avant_dernier_poi, dernier_poi,
            row_number() over (partition by avant_dernier_poi, dernier_poi order by jour desc) as num
            from trajectoires_eligibles        
            """
    return spark.sql(query)

only one trajectory per suffix: the most recent

In [108]:
def trajectoire_eligible_par_suffixe(trajectoires_eligibles_num):
    trajectoires_eligibles_num.createOrReplaceTempView("trajectoires_eligibles_num")
    query = """
            select  t.user, t.jour, t.trajectoire,avant_dernier_poi, dernier_poi
            from trajectoires_eligibles_num t
            where num=1
            """
    return spark.sql(query)

#### Create train trajectories

In [110]:
def trajectoire_de_train(table_trajectoires_superieur_3, table_trajectoires_test):
    table_trajectoires_superieur_3.createOrReplaceTempView("trajectoires_sup3")
    table_trajectoires_test.createOrReplaceTempView("traj_test")
    
    
    query = """
            select user, jour, trajectoire
            from trajectoires_sup3
            minus
            select user, jour, trajectoire
            from 
            traj_test;
            
            """
    return spark.sql(query)

#### get test and train  set

In [112]:
def get_or_create_testset_and_trainset(PUBLIC_DATASET_URL, yfcc_web_dir, local_dir, yfcc_file, n):
    
    test_set_file = local_dir + "/" + yfcc_file + f"_M_{n}_testset"
    train_set_file = local_dir + "/" + yfcc_file + f"_M_{n}_trainset"
    
    if not ( os.path.exists(test_set_file) and  os.path.exists(train_set_file) ) :
        yfcc_web_dir_download_URL = PUBLIC_DATASET_URL + "/download?path=" + "/" + yfcc_web_dir
        get_yfcc_dataset(yfcc_web_dir_download_URL , local_dir, yfcc_file)

        yfcc = lire_yfcc(yfcc_file)
        visite1 = visite(yfcc)visiteRound = visit_round_to_n_meters(visite1, n)
        frequence = lat_long_freq(visiteRound)
        latlongFrance = bounding_box_France(frequence)
        table_avec_num_POI = numorer_POI(latlongFrance)
        table_visite_avec_num_POI = visiteNum(table_avec_num_POI, visiteRound)
        traj_toutes_tailles = toutes_tailles_trajectoires(table_visite_avec_num_POI)
        trajectoires_superieur_3 = trajectoires_sup3(traj_toutes_tailles)
        trajectoires_eligibles = trajectoire_eligible(trajectoires_superieur_3)
        trajectoires_eligibles_num = trajectoire_eligible_numerotation(trajectoires_eligibles)
        trajectoires_eligibles_num = trajectoire_eligible_numerotation(trajectoires_eligibles)
        trajectoire_test = trajectoire_eligible_par_suffixe(trajectoires_eligibles_num)
        trajectoires_train = trajectoire_de_train(trajectoires_superieur_3, trajectoire_test)
        
        # write trajectoire_test
        trajectoire_test.write.mode("overwrite").format("parquet").save(test_set_file)
        trajectoires_train.write.mode("overwrite").format("parquet").save(train_set_file)   
    else:
        print("test set or train set already exist")
        
    return spark.read.format("parquet").load(test_set_file).persist(), spark.read.format("parquet").load(train_set_file).persist()

#### Length of all the trajectories (sup3, train, test)

In [113]:
liste_toutes_valeurs_compt = []
for n in P.maillage:
    yfcc = lire_yfcc(P.yfcc_file)
    visite1 = visite(yfcc)
    visiteRound = visit_round_to_n_meters(visite1, n)
    frequence = lat_long_freq(visiteRound)
    
    latlongFrance = bounding_box_France(frequence)
    nombre_poi_france = latlongFrance.count()
    
    table_avec_num_POI = numorer_POI(latlongFrance)
    table_visite_avec_num_POI = visiteNum(table_avec_num_POI, visiteRound)
    traj_toutes_tailles = toutes_tailles_trajectoires(table_visite_avec_num_POI)
    
    trajectoires_superieur_3 = trajectoires_sup3(traj_toutes_tailles)
    nombre_trajectories_sup3 = trajectoires_superieur_3.count()
    
    trajectoires_eligibles = trajectoire_eligible(trajectoires_superieur_3)
    trajectoires_eligibles_num = trajectoire_eligible_numerotation(trajectoires_eligibles)

    trajectoire_test = trajectoire_eligible_par_suffixe(trajectoires_eligibles_num)
    nombre_trajectories_test = trajectoire_test.count()

    trajectoires_train = trajectoire_de_train(trajectoires_superieur_3, trajectoire_test)
    nombre_trajectoires_train = trajectoires_train.count()

    longueur_moyenne_traj_test = (trajectoire_test.select("trajectoire").withColumn("longueur", f.size("trajectoire"))).agg(f.avg("longueur").alias("longueur_moyenne"))
    longueur_moyenne_traj_test_pandas = longueur_moyenne_traj_test.toPandas()
    longueur_moyenne_traj_test_par_maillage = longueur_moyenne_traj_test_pandas["longueur_moyenne"][0]

    tuple_taille_traj_par_maillage = (n, nombre_poi_france, nombre_trajectories_sup3, nombre_trajectories_test, nombre_trajectoires_train, longueur_moyenne_traj_test_par_maillage)
    liste_toutes_valeurs_compt.append(tuple_taille_traj_par_maillage)
    
columns=['maillage','nombre_poi_france','nombre_trajectoire_sup3','nombre_trajectories_test','nombre_trajectoires_train','longueur_moyenne_seq_test_par_maillage'] 
nbr_traj_train_test= pd.DataFrame(liste_toutes_valeurs_compt, columns = columns)
nbr_traj_train_test

,maillage,nombre_poi_france,nombre_trajectoire_sup3,nombre_trajectories_test,nombre_trajectoires_train,longueur_moyenne_seq_test_par_maillage
0,10,473296,65731,235,65422,4.970213
1,40,326923,65491,863,64535,6.028969
2,70,266331,64373,1602,62671,5.865169
3,150,194579,60870,3007,57766,5.371467
4,300,142211,54833,3900,50798,4.766923


#### Execute the creation of the test sets and train

In [117]:
print("URL du dossier contenant les datasets ", P.PUBLIC_DATASET_URL)
for m in P.maillage:
    print(m)
    trajectoires_test, trajectoires_train = get_or_create_testset_and_trainset(P.PUBLIC_DATASET_URL, P.yfcc_web_dir, P.local_dir, P.yfcc_file, m)

URL du dossier contenant les datasets  https://nuage.lip6.fr/s/H3bpyRGgnCq2NR4
10
test set or train set already exist
40
test set or train set already exist
70
test set or train set already exist
150
test set or train set already exist
300
/local/jarrad/bd//yfccFrance.zip is already stored
file already unziped


## Models creation

### EMBEDDINGS

In [119]:
def get_input_trajectoires(trajectoires_train): 
    input_trajectoires_1 = trajectoires_train.selectExpr("trajectoire")
    input_trajectoires = [ [str(poi) for poi in t.trajectoire] for t in input_trajectoires_1.collect()]
    
    return input_trajectoires

In [120]:
def get_model_name_dim(model_name, dimension, epoch, window):
    return model_name + "_dimension" + str(dimension) + "_epoch" +str(epoch)+ "_window" +str(window)


In [121]:
def dimension_model(trajectoires_train, model_name, expe_dimensions, expe_epochs, expe_windows):

    input_trajectoires = get_input_trajectoires(trajectoires_train)
    for dimension in expe_dimensions:
        for epoch in expe_epochs:
            for window in expe_windows:
    
                model_to_save = get_model_name_dim(model_name, dimension, epoch, window)
                #si le modele n'existe pas deja
                if not os.path.exists(model_to_save) :
                    model = Word2Vec(sentences=input_trajectoires, vector_size=dimension, epochs =epoch, window=window, min_count=1, workers=64)
                    model.save(model_to_save)
                    print("model_saved" , model_to_save)
                else:
                    print("model already exists" , model_to_save)
            

### Execute models creation

In [122]:
for m in P.maillage:
    model_name = P.model_name + f"_M{m}"
    print(m)
    trajectoires_test, trajectoires_train = get_or_create_testset_and_trainset(P.PUBLIC_DATASET_URL, P.yfcc_web_dir, P.local_dir, P.yfcc_file, m)
    dimension_model(trajectoires_train, model_name, P.expe_dimensions, P.expe_epochs, P.expe_windows)

10
test set or train set already exist
model_saved /local/jarrad/bd/model_dimension/model_M10_dimension20_epoch100_window5
model already exists /local/jarrad/bd/model_dimension/model_M10_dimension25_epoch100_window5
40
test set or train set already exist
model_saved /local/jarrad/bd/model_dimension/model_M40_dimension20_epoch100_window5
model already exists /local/jarrad/bd/model_dimension/model_M40_dimension25_epoch100_window5
70
test set or train set already exist
model_saved /local/jarrad/bd/model_dimension/model_M70_dimension20_epoch100_window5
model already exists /local/jarrad/bd/model_dimension/model_M70_dimension25_epoch100_window5
150
test set or train set already exist
model_saved /local/jarrad/bd/model_dimension/model_M150_dimension20_epoch100_window5
model already exists /local/jarrad/bd/model_dimension/model_M150_dimension25_epoch100_window5
300
test set or train set already exist
model_saved /local/jarrad/bd/model_dimension/model_M300_dimension20_epoch100_window5
model_sa

# Prediction algorithms

## similarity between trajectories

In [124]:
def similarite_trajectoires_JACCARD(traj_test, trajectoire):
    s1 = set(traj_test[:-1])
    s2 = set(trajectoire[:-1])
    sim = len(s1.intersection(s2)) / len(s1.union(s2))
    return sim


# Mean reciprocal rank MRR
def similarite_trajectoires_MRR(traj_test, trajectoire):
    somme = 0.0
    for i in range(min(len(traj_test) , len(trajectoire))-1):
        if traj_test[-2-i] == trajectoire[-2-i]:
             somme += 1 / (i+1)    
    return somme

def load_w2v_model(model_name, dimension, epoch, window): 
    global MODEL
    return MODEL   


def similarite_trajectoires_JACCARD_cosinus(traj_test, trajectoire, model_name, dimension, epoch, window):
    model = load_w2v_model( model_name, dimension, epoch, window)
    somme = 0.0
    for p1 in traj_test[:-1]:
        try:
            v1 = model.wv[str(p1)]
            max_sim = -2
            for p2 in trajectoire[:-1]:
                v2 = model.wv[str(p2)]
                sim = dot(v1, v2)/(norm(v1)*norm(v2))
                #if sim > 0.6:
                max_sim = max(max_sim, sim)

        except:
            continue
        somme += max_sim
            
    return float(somme)
def similarite_trajectoires_MRR_cosinus(traj_test, trajectoire, model_name, dimension, epoch, window):
    model = load_w2v_model( model_name, dimension, epoch, window)
    somme = 0.0
    for i in range(min(len(traj_test) , len(trajectoire))-1):
        try:
            v1 = model.wv[str(traj_test[-2-i])]
            max_sim = -2
            for p2 in trajectoire[:-1]:
                v2 = model.wv[str(p2)]
                sim = dot(v1, v2)/(norm(v1)*norm(v2))
                #if sim > 0.6:
                max_sim = max(max_sim, sim)
        except:
            continue
        somme += max_sim / (i+1)   
    return float(somme)

In [130]:
def sim_traj(traj_test, trajectoire_train, fct_similarite, model_name, dimension, epoch, window):
    if fct_similarite=="similarite_trajectoires_JACCARD": 
        sim = similarite_trajectoires_JACCARD(traj_test, trajectoire_train)

    elif fct_similarite=="similarite_trajectoires_MRR": 
        sim = similarite_trajectoires_MRR(traj_test, trajectoire_train)

    elif fct_similarite=="similarite_trajectoires_JACCARD_cosinus": 
        sim = similarite_trajectoires_JACCARD_cosinus(traj_test, trajectoire_train, model_name, dimension, epoch, window)
        
    elif fct_similarite=="similarite_trajectoires_MRR_cosinus": 
        sim = similarite_trajectoires_MRR_cosinus(traj_test, trajectoire_train, model_name, dimension, epoch, window)
    else:
        print("erreur fonction inconnue", fct_similarite)
    return sim
         
spark.udf.register("sim_traj", sim_traj, DoubleType())

<function __main__.sim_traj(traj_test, trajectoire_train, fct_similarite, model_name, dimension, epoch, window)>

## paires de trajectoires 

## pairs of trajectories 
test/train that have the LAST POI in common

In [133]:
def paire_trajectoires(table_traj_train, table_traj_test):
    table_traj_train.createOrReplaceTempView("traj_de_train")
    table_traj_test.createOrReplaceTempView("traj_de_test")
  
    
    query = f"""
           SELECT test.trajectoire as traj_test, 
                  train.trajectoire as traj_train
            FROM traj_de_test test , traj_de_train train
            WHERE element_at(test.trajectoire, -2) = element_at(train.trajectoire, -2) 
            ;
            """
    return spark.sql(query)

In [150]:
#Experience 3 : Toutes les trajectories train/test qui ont au moins 1 point en commun à n'importe quelle position

#def paire_trajectoires(table_traj_train, table_traj_test):
#    table_traj_train.createOrReplaceTempView("traj_de_train")
#    table_traj_test.createOrReplaceTempView("traj_de_test")

#    query = f"""
#           SELECT test.trajectoire as traj_test, 
#                  train.trajectoire as traj_train
#            FROM traj_de_test test , traj_de_train train
#            WHERE intersection_entre_trajectoires(test.trajectoire, train.trajectoire) >0
#            ;
#            """
#    return spark.sql(query)

## Trajctories ranking

For each test trajectory, ranking of training trajectories by decreasing similarity

In [151]:
def classement_trajectoires(table_paires_trajectoires, fonction_similarite, dimension, model_name, epoch, window):
    table_paires_trajectoires.createOrReplaceTempView("table_paires_trajectoire")
    
    
    query = f"""
            select *, sim_traj(traj_test, traj_train, {fonction_similarite}, '{model_name}', {dimension} , {epoch}, {window}) as similarite,
            rank() over( partition by traj_test order by sim_traj(traj_test, traj_train, {fonction_similarite}, '{model_name}', {dimension}, {epoch}, {window} ) desc) as classement
            from table_paires_trajectoire
            order by traj_test, similarite desc;
            
            """
    return spark.sql(query)

Group the TOP k most similar trajectories (for each test trajectory) by last train POI

In [154]:
def classement_top_k_traj_les_plus_sim(table_classement_des_trajectoires_par_sim, k):
    table_classement_des_trajectoires_par_sim.createOrReplaceTempView("similarite_par_trajectoire_par_dernier_POI_du_train") 
    query = f"""
            select traj_test, element_at(traj_train, -1) as dernier_poi_train, sum(similarite) as similarite
            from similarite_par_trajectoire_par_dernier_POI_du_train
            where classement <= {k}
            group by traj_test, element_at(traj_train, -1)
            order by traj_test, similarite desc            
            """ 
    return spark.sql(query)

## Best score

keep only the prediction with the best score

In [156]:
def pred_avec_meilleur_score(table_top_k_classement_traj_similaires):
    table_top_k_classement_traj_similaires.createOrReplaceTempView("trajectoire_avec_prediction")
    
    
    query = """

            select traj_test, element_at(array_sort(collect_list( (similarite, dernier_poi_train))), -1) as prediction
            from trajectoire_avec_prediction
            group by traj_test;
    
            
            
            """
    return spark.sql(query)

## prediction

In [158]:
def prediction(table_prediction_meilleur_score):
    table_prediction_meilleur_score.createOrReplaceTempView("trajectoire_avec_meilleure_prediction")
    
    
    query = """

            select traj_test, element_at(traj_test, -1) as testPOI, prediction.dernier_poi_train as prediction
            from trajectoire_avec_meilleure_prediction;
    
            
            
            """
    return spark.sql(query)

## prediction success rate

In [160]:
def taux_succes_prediction(table_prediction):
    table_prediction.createOrReplaceTempView("prediction_point")
   
   
   
    query1 = """
           
            select count(*)  as nb from prediction_point where testPOI = prediction;

            """
   
    query2 = """select count(*)  as nb from prediction_point """
   
    succes = spark.sql(query1)
    succes_pandas = succes.toPandas()
    success = succes_pandas["nb"][0]
    #print(success)
   
    total = spark.sql(query2)
    total_pandas = total.toPandas()
    totall = total_pandas["nb"][0]
    #print(totall)
   
    return success/totall

# EXPERIENCES

## Configurable experiences

## Quality

In [162]:
def mesure_qualite(paires_avec_similarite_decroissante, k):
    top_k_classement_traj_similaires = classement_top_k_traj_les_plus_sim(paires_avec_similarite_decroissante, k)
    prediction_avec_meilleur_score = pred_avec_meilleur_score(top_k_classement_traj_similaires)
    table_prediction = prediction(prediction_avec_meilleur_score)
    taux = taux_succes_prediction(table_prediction)
    return taux

In [164]:
def toutes_experiences(expe_topk, expe_dimensions,expe_epochs, model_name, experiment_dir, expe_maillage, expe_windows):
    global MODEL

    # name of the file in which the results of experiments based on embeddings are stored
    def get_expe_vector_filemane(i):
        return  experiment_dir + "/" + f"expe_{i}_vectors.csv"
    
    
    # name of the file in which the results of experiments non based embeddings are stored
    def get_expe_no_vector_filemane(i):
        return  experiment_dir + "/" + f"expe_{i}_no_vectors.csv"
    
    file_num=1
    while os.path.exists(get_expe_vector_filemane(file_num)) or os.path.exists(get_expe_no_vector_filemane(file_num)):
        file_num = file_num + 1
    with open(get_expe_vector_filemane(file_num), 'a') as f:
        print("resultats expe avec vecteurs W2V dans :",experiment_dir + get_expe_vector_filemane(file_num))
        f.write('maillage,k,dimension,epoch,window,fonction de similarite,qualite\n')
        
    with open(get_expe_no_vector_filemane(file_num), 'a') as f_sans_model:
        print("resultats expe sans vecteurs dans :",experiment_dir + get_expe_no_vector_filemane(file_num))
        f_sans_model.write('maillage,k,dimension,epoch,window,fonction de similarite,qualite\n')
        
    
    for m in P.maillage:
        trajectoires_test, trajectoires_train = get_or_create_testset_and_trainset(P.PUBLIC_DATASET_URL, P.yfcc_web_dir, P.local_dir, P.yfcc_file, m)
        paires = paire_trajectoires(trajectoires_train, trajectoires_test)
        paires.persist()
        
        
        # with embeddings
        with open(get_expe_vector_filemane(file_num), 'a') as f:
            for dimension in expe_dimensions:
                for epoch in expe_epochs:
                    for window in expe_windows:
                        model_name = P.model_name + f"_M{m}"
                        model_to_load = get_model_name_dim(model_name, dimension, epoch, window)
                        MODEL = Word2Vec.load(model_to_load)
                        spark.udf.register("sim_traj", sim_traj, DoubleType())
                        for fonction_similarite in ('\"similarite_trajectoires_MRR_cosinus\"', '\"similarite_trajectoires_JACCARD_cosinus\"'):
                            paires_avec_sim = classement_trajectoires(paires, fonction_similarite, dimension, model_name, epoch, window)
                            paires_avec_sim.persist()
                            for k in expe_topk:
                                qualite = mesure_qualite(paires_avec_sim, k)
                                tuple_experience = (m, k, dimension, epoch, window, fonction_similarite, qualite)
                                sep = ','
                                f.write(str(m) +sep + str(k) + sep + str(dimension) + sep + str(epoch)+ sep + str(window) + sep + str(fonction_similarite) + sep + str(qualite) + '\n')
                                f.flush()

                        
    
    
        # without embeddings
        with open(get_expe_no_vector_filemane(file_num), 'a') as f_sans_model:
            for fonction_similarite in ('\"similarite_trajectoires_JACCARD\"', '\"similarite_trajectoires_MRR\"'):
                # calculer la similarité + tri
                paires_avec_sim = classement_trajectoires(paires, fonction_similarite, dimension, model_name, epoch, window)
                paires_avec_sim.persist()

                for k in expe_topk:
                    qualite = mesure_qualite(paires_avec_sim, k)
                    tuple_experience = (m,k, 0,0,0, fonction_similarite, qualite)
                    sep = ','
                    f_sans_model.write(str(m) + sep +str(k) + sep + "0" + sep + "0"+ sep + "0"+ sep +str(fonction_similarite) + sep + str(qualite) + '\n')
                    f_sans_model.flush()


## Lancer les expériences

In [165]:
from datetime import datetime
t1 = datetime.now()
print(t1)

toutes_experiences(expe_topk = P.expe_topk, expe_dimensions= P.expe_dimensions, expe_epochs=P.expe_epochs,model_name = P.model_name, experiment_dir=P.experiment_dir, expe_maillage=P.maillage, expe_windows=P.expe_windows)

t2 = datetime.now()
print(t2, "duree:", t2-t1)

2023-05-01 23:26:43.257976
resultats expe avec vecteurs W2V dans : /home/jarrad/Bureau/notebook_sara/experiment//home/jarrad/Bureau/notebook_sara/experiment//expe_185_vectors.csv
resultats expe sans vecteurs dans : /home/jarrad/Bureau/notebook_sara/experiment//home/jarrad/Bureau/notebook_sara/experiment//expe_185_no_vectors.csv
test set or train set already exist
test set or train set already exist
test set or train set already exist
test set or train set already exist
test set or train set already exist
2023-05-01 23:33:24.730624 duree: 0:06:41.472648
